# **Hit Song Attributes**

# Project: Milestone 4

## Connecting to an API/Pulling in the Data and Cleaning/Formatting

Amelia Farrell

DSC540

5/14/23

In [5]:
# Connecting to google drive
from google.colab import drive 
drive.mount('/content/drive')

# Change to the google drive directory
%cd /content/drive/MyDrive/Colab\ Notebooks/DSC540/FinalProject

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/DSC540/FinalProject


In [35]:
!pip install ratelimit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5893 sha256=9f7c8e754f167fe25fafc9585be388ef9e8fdf7788739acbb2c88d2cb553cf54
  Stored in directory: /root/.cache/pip/wheels/27/5f/ba/e972a56dcbf5de9f2b7d2b2a710113970bd173c4dcd3d2c902
Successfully built ratelimit


In [36]:
# importing libraries needed
import requests
import json
import pandas as pd
from ratelimit import limits, RateLimitException, sleep_and_retry

## Step 1: Web & Flat File

Before we can call the API, we need to know what songs we need date information for. Since our API contains thousands of songs, we do not want to pull ALL the data. But how do we know what songs to gather dates for? This will be based on the song list we obtained from the Web (wikipedia top songs). Since the Website data does not have he spotify track ID that we need for the API, we will also add combine the web data with the Flat File. Therefore, our first step will be to load and combine the Web Data and Flat File. 

Since we have both of these data sets saved as a csv, we will load each and combine them (getting the list of only the spotify track IDs we need).

In [32]:
# loading final flat file
flat_file_final = pd.read_csv('flat_file_final.csv')
lenf = len(flat_file_final)
# loading final website data
web_data_final = pd.read_csv('web_data_final.csv')
lenw = len(web_data_final)
# print difference in length
print(lenf - lenw)

39910


In [28]:
# viewing the flat file
flat_file_final.head()

,song_name,song_artist,spotify_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,uid
0,jealous kind of fella,garland green,spotify:track:1dtKN6wwlolkM8XZy2y9C1,0.417,0.620,3.0,-7.727,1.0,0.0403,0.490,0.000000,0.0779,0.845,185.655,173533.0,3.0,32.94975,9.0,e8f3efdd6ac74332876f615af0daf6c7
1,initials b.b.,serge gainsbourg,spotify:track:5hjsmSnUefdUqzsDogisiX,0.498,0.505,3.0,-12.475,1.0,0.0337,0.018,0.107000,0.1760,0.797,101.801,213613.0,4.0,48.82510,10.0,8757b103a9738f17425e455bdfe7242a
2,melody twist,lord melody,spotify:track:6uk8tI6pwxxdVTNlNOJeJh,0.657,0.649,5.0,-13.392,1.0,0.0380,0.846,0.000004,0.1190,0.908,115.940,223960.0,4.0,37.22663,12.0,dc2354b2794a897927eda04b5f575413
3,mi bomba sonó,celia cruz,spotify:track:7aNjMJ05FvUXACPWZ7yJmv,0.590,0.545,7.0,-12.058,0.0,0.1040,0.706,0.024600,0.0610,0.967,105.592,157907.0,4.0,24.75484,8.0,a50a8e15cab1e07b83e1726d9ca4eda8
4,uravu solla,p. susheela,spotify:track:1rQ0clvgkzWr001POOPJWx,0.515,0.765,11.0,-3.515,0.0,0.1240,0.857,0.000872,0.2130,0.906,114.617,245600.0,4.0,21.79874,14.0,d8c645edfe864872c257a70d7f527e97


In [29]:
# viewing the website data
web_data_final.head()

,year,song_name,song_artist,uid
0,1946,prisoner of love,perry como,5ffa72f4e33500df9151c0ca490e9758
1,1947,near you,francis craig,a765d8bda8572ac7980ec49d5d645f15
2,1948,twelfth street rag,pee wee hunt,72f21b83776b80770dae54606fff03d0
3,1949,riders in the sky,vaughn monroe orchestra,8c3661637a4ab864521e13a379b16b17
4,1950,"goodnight, irene",gordon jenkins and the weavers,51abcf5725b1806fb1fdf89cabc0a7bd


In [47]:
# dropping extra song and artist columns from website data
web_data_final.drop(['song_name', 'song_artist'], axis=1, inplace=True)

In [48]:
# combinding the website data and flat file
df_merged = flat_file_final.merge(web_data_final, left_on=['uid'], right_on=['uid'])
lenm = len(df_merged)
print(lenm)

77


In [49]:
# viewing combined file
df_merged.head()

,song_name,song_artist,spotify_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,uid,year
0,sugar shack,jimmy gilmer and the fireballs,spotify:track:2nAR2OjaGIXesjm7Kq7Kzu,0.856,0.755,7.0,-8.550,1.0,0.1830,0.2210,0.000279,0.104,0.805,132.382,121800.0,4.0,34.23680,8.0,84ec21311fc6b78aa0d04b4f5d6e37d0,1963
1,"sugar, sugar",the archies,spotify:track:3iSws76HjaU7k49EqJVTfF,0.736,0.871,2.0,-2.786,1.0,0.0278,0.4660,0.000000,0.108,0.967,122.395,167187.0,4.0,34.20599,9.0,43c6d0a204491796779ab56ab22f844a,1969
2,hey jude,the beatles,spotify:track:0aym2LBJBk9DAYuHHutrIl,0.386,0.607,10.0,-7.700,1.0,0.0261,0.0112,0.000014,0.088,0.532,147.207,425653.0,4.0,26.10728,21.0,3daef7ace417cf0ba5df256f2893ce9b,1968
3,tossin' and turnin',bobby lewis,spotify:track:79s3pSe1u6ggQBEwVPm58Z,0.569,0.722,0.0,-8.914,1.0,0.0393,0.2250,0.000000,0.108,0.965,142.736,159640.0,4.0,51.05837,8.0,957a2ba9e3b78209f2a31f50cdcc0f5a,1961
4,i want to hold your hand,the beatles,spotify:track:4pbG9SUmWIvsROVLF0zF9s,0.490,0.715,7.0,-5.549,1.0,0.0476,0.3860,0.000000,0.311,0.866,130.726,145747.0,4.0,27.48807,8.0,09e1284b4af72586ec497713397fb7e5,1964


In [72]:
# dropping extra columns from combined data
df_merged.drop(['danceability', 'energy','key', 'loudness','mode', 'speechiness','acousticness', 'instrumentalness', 'liveness','valence', 'tempo','duration_ms', 'time_signature','chorus_hit', 'sections','year'], axis=1, inplace=True)

## Step 2: Function

Now that we have a list of all the Keys needed for the API, we can use them to pull the release date for each song. To do so, we need to create a function to call the API for each line of our data and return the 'album_release_date'.

In [ ]:
# creating our function
@sleep_and_retry
@limits(calls=5, period=1) #API Limit of 5 calls per second
def get_album_data(row):

  url = "https://spotify23.p.rapidapi.com/tracks/"

  querystring = {"ids":row["spotify_id"].split(":")[2]}

  headers = {
    "X-RapidAPI-Key": "9d0e146c26mshf2a9609c55e0254p181d7fjsn9a211d034509",
    "X-RapidAPI-Host": "spotify23.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring)
  response = json.loads(response.text)

  row['album_release_date'] = response['tracks'][0]['album']['release_date']
  row['album_name'] = response['tracks'][0]['album']['name']
  row['album_id'] = response['tracks'][0]['album']['uri']

  return row


In [73]:
# running function on our merged website and flat file data
final_df = df_merged.apply(get_album_data, axis=1)

## Step 3: View Data

Now that we pulled the release date for each of our songs from the API, lets ensure that it shows in our final dataset. 

In [78]:
final_df.head()

,song_name,song_artist,spotify_id,uid,album_release_date,album_name,album_id
0,sugar shack,jimmy gilmer and the fireballs,spotify:track:2nAR2OjaGIXesjm7Kq7Kzu,84ec21311fc6b78aa0d04b4f5d6e37d0,2009-10-01,Clovis Classics: The Definitive Collection,spotify:album:1AD44MeoSIgCJNcPAFCSCo
1,"sugar, sugar",the archies,spotify:track:3iSws76HjaU7k49EqJVTfF,43c6d0a204491796779ab56ab22f844a,2007-11-07,"The Very Best Of ""The Archies""",spotify:album:7spOTrIPTM8RrMIAhCjLJi
2,hey jude,the beatles,spotify:track:0aym2LBJBk9DAYuHHutrIl,3daef7ace417cf0ba5df256f2893ce9b,2000-11-13,1 (Remastered),spotify:album:7vEJAtP3KgKSpOHVgwm3Eh
3,tossin' and turnin',bobby lewis,spotify:track:79s3pSe1u6ggQBEwVPm58Z,957a2ba9e3b78209f2a31f50cdcc0f5a,2005-01-01,Tossin' and Turnin',spotify:album:31JyXxhju4kXiYJwdjb3u3
4,i want to hold your hand,the beatles,spotify:track:4pbG9SUmWIvsROVLF0zF9s,09e1284b4af72586ec497713397fb7e5,2000-11-13,1 (Remastered),spotify:album:7vEJAtP3KgKSpOHVgwm3Eh


## Step 4: Modify Data

Since our data is not in date format, we will convert it to a date. We will then be able to create a month and year column for each of the songs. This will give us the data needed to look at seasonality as well as the gaps between the year it was released (date from the API) and the year it became a ‘Hit’(date from the Web).


In [79]:
# checking the data type of our new Release date column from the API
final_df.dtypes['album_release_date']

dtype('<M8[ns]')

In [80]:
# converting album_release_date to date
final_df['album_release_date'] = pd.to_datetime(final_df['album_release_date'])
final_df.dtypes['album_release_date']

dtype('<M8[ns]')

In [81]:
# converting album_release_date to month
album_release_month = pd.to_datetime(final_df['album_release_date'], format='%m').dt.strftime('%B')

# converting album_release_date to year
album_release_year = pd.to_datetime(final_df['album_release_date']).dt.year

In [82]:
# adding album_release_month to dataframe
final_df['album_release_month'] = album_release_month

In [83]:
# adding album_release_year to dataframe
final_df['album_release_year'] = album_release_year

In [84]:
# veiw dataset with new columns
final_df.head()

,song_name,song_artist,spotify_id,uid,album_release_date,album_name,album_id,album_release_month,album_release_year
0,sugar shack,jimmy gilmer and the fireballs,spotify:track:2nAR2OjaGIXesjm7Kq7Kzu,84ec21311fc6b78aa0d04b4f5d6e37d0,2009-10-01,Clovis Classics: The Definitive Collection,spotify:album:1AD44MeoSIgCJNcPAFCSCo,October,2009
1,"sugar, sugar",the archies,spotify:track:3iSws76HjaU7k49EqJVTfF,43c6d0a204491796779ab56ab22f844a,2007-11-07,"The Very Best Of ""The Archies""",spotify:album:7spOTrIPTM8RrMIAhCjLJi,November,2007
2,hey jude,the beatles,spotify:track:0aym2LBJBk9DAYuHHutrIl,3daef7ace417cf0ba5df256f2893ce9b,2000-11-13,1 (Remastered),spotify:album:7vEJAtP3KgKSpOHVgwm3Eh,November,2000
3,tossin' and turnin',bobby lewis,spotify:track:79s3pSe1u6ggQBEwVPm58Z,957a2ba9e3b78209f2a31f50cdcc0f5a,2005-01-01,Tossin' and Turnin',spotify:album:31JyXxhju4kXiYJwdjb3u3,January,2005
4,i want to hold your hand,the beatles,spotify:track:4pbG9SUmWIvsROVLF0zF9s,09e1284b4af72586ec497713397fb7e5,2000-11-13,1 (Remastered),spotify:album:7vEJAtP3KgKSpOHVgwm3Eh,November,2000


## Step 5: Ethical Implications

 When thinking about the ethical implications of our spotify API data gathering, it helps to take another look at the Harvard Business School 5 principles of data ethics. Ownership, Transparency, Privacy, Intention, and Outcomes (5 Principles of Data Ethics for Business, 2021). Does our data gathering method align with these guidelines? How does it conform to the 'ownership' principle? Since we do not own these songs, nor the data tied to them...

"The first principle of data ethics is that an individual has ownership over their personal information" (5 Principles of Data Ethics for Business, 2021). Since our data does not contain personal information, our methods would be considered 'ethical' in regards to the 'ownership' principle.

What about Transparency, Privacy, Intention, and Outcomes? Luckily none of our data deals with personal information so we are also in the clear in regards to the next three. But what about Outcomes?

"Even when intentions are good, the outcome of data analysis can cause inadvertent harm to individuals or groups of people." (5 Principles of Data Ethics for Business, 2021). This is the principle that can be difficult to foresee. However, even before completing our analysis it is best practice to make some assumptions on how the data could be viewed and what impact it could have. To tie back to our API data, one poor outcome would be the chance we gathered incorrect data. We did not double check the release dates against any other data source. So how do we know they are correct? This is certainly something that could come up in the final analysis or even be realized after it is released. To avoid issues such as this we should always vaildate the data against another source if possible. 




*5 Principles of Data Ethics for Business. (2021, March 16). Business Insights Blog. https://online.hbs.edu/blog/post/data-ethics*